Initial Setup

In [1]:
import os
import tensorflow

2024-05-09 23:41:34.434785: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-09 23:41:34.437006: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-09 23:41:34.461318: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 23:41:34.461354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 23:41:34.462643: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

# LabelMaps

## Creacion de Labelmaps

In [4]:
!python ./csv_to_labelmap.py

2024-05-09 23:41:36.637907: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-09 23:41:36.639109: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-09 23:41:36.662832: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 23:41:36.662892: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 23:41:36.663839: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

# TFRecords

## Train Model

In [5]:
!python ./generate_tfRecord.py "Playing Cards.v4-yolov8n.tensorflow/train/_annotations.csv" label_map.pbtxt "Playing Cards.v4-yolov8n.tensorflow/train" train.record

2024-05-09 23:41:39.424212: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-09 23:41:39.425912: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-09 23:41:39.449565: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 23:41:39.449618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 23:41:39.450182: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

## Test Model

In [6]:
!python ./generate_tfRecord.py "Playing Cards.v4-yolov8n.tensorflow/test/_annotations.csv" label_map.pbtxt "Playing Cards.v4-yolov8n.tensorflow/test" test.record

2024-05-09 23:44:02.178775: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-09 23:44:02.180457: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-09 23:44:02.200909: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 23:44:02.200962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 23:44:02.201583: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

## Valid Model

In [7]:
!python ./generate_tfRecord.py "Playing Cards.v4-yolov8n.tensorflow/valid/_annotations.csv" label_map.pbtxt "Playing Cards.v4-yolov8n.tensorflow/valid" valid.record

2024-05-09 23:44:12.334147: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-09 23:44:12.335649: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-09 23:44:12.355476: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 23:44:12.355515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 23:44:12.356102: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

# MLModel

## Libraries

In [6]:
import wget
import tarfile
import os
import re

## Enviroment Settings

In [7]:
#Can be configured
MODEL_NAME = "faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8"
MODEL_LINK = "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz"
MODEL_CONFIG = "faster_rcnn_inception_v2_coco.config"
MODELS_FOLDER = os.path.join('/tf','tensorflow','models','research','object_detection','samples','configs')
MODEL_FILEPATH = os.path.join(MODELS_FOLDER, MODEL_CONFIG)

In [8]:
FINE_CHECKPOINT = os.path.join('models', MODEL_NAME, 'checkpoint')
TRAIN_TFRECORD = os.path.join('train.record')
TEST_TFRECORD = os.path.join('test.record')
LABEL_MAP = os.path.join('label_map.pbtxt')

In [9]:
BATCH_SIZE = 12
NUM_STEPS = 10000
NUM_CLASSES = 52
NUM_EVAL_STEPS = 50

## Downloading ML Model

In [13]:
wget.download(MODEL_LINK, out='./models')

'./models/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz'

## Unzip MLModel

In [14]:
tar = tarfile.open('./models/' + MODEL_NAME + ".tar.gz")
tar.extractall('./models/')
tar.close()

## Modify config file


In [6]:
with open(MODEL_FILEPATH) as f:
    s = f.read()

with open(MODEL_FILEPATH, 'w') as f:
        # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(FINE_CHECKPOINT), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(TRAIN_TFRECORD), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(TEST_TFRECORD), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(LABEL_MAP), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(BATCH_SIZE), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(NUM_STEPS), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(NUM_CLASSES), s)
    f.write(s)

In [10]:
!cat {MODEL_FILEPATH}

# Faster R-CNN with Inception v2, configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.


model {
  faster_rcnn {
    num_classes: 52
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncate

## Training model

In [16]:
!python /tf/tensorflow/models/research/object_detection/model_main.py \
    --num_train_steps={NUM_STEPS}

2024-05-10 17:51:56.075532: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 17:51:56.076716: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-10 17:51:56.096283: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 17:51:56.096324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 17:51:56.096977: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to